# GMSPY demo

In [1]:
import gmspy
import gams
import pandas as pd
import numpy as np

## Define some Data

In [2]:
# 3 sets
cohorts = [str(i) for i in range(2020, 2029)] # Important that this be string, not integers
tech = ['bev', 'icev']
imps = ['gwp']

In [3]:
# total demand - pd Series
Y = pd.Series(index=tech, data=[100, 2])
Y

bev     100
icev      2
dtype: int64

In [4]:
# Stock - 2d DataFrame
STOCK = pd.DataFrame(index=tech, columns=cohorts)
STOCK[:] = np.random.rand(*STOCK.shape)
STOCK

,2020,2021,2022,2023,2024,2025,2026,2027,2028
bev,0.688669,0.079602,0.594031,0.011032,0.662522,0.422199,0.795230,0.197732,0.149490
icev,0.103520,0.351881,0.144436,0.342146,0.679196,0.471649,0.004622,0.812067,0.714773


In [5]:
# IMP - 3d DataFrame
ix = pd.MultiIndex.from_product((tech, imps))
IMP = pd.DataFrame(index=ix, columns=cohorts)
IMP[:] = np.random.rand(*IMP.shape)
IMP

,,2020,2021,2022,2023,2024,2025,2026,2027,2028
bev,gwp,0.665257,0.996560,0.575023,0.224564,0.197891,0.298945,0.981139,0.400548,0.001846
icev,gwp,0.414993,0.799778,0.410036,0.020946,0.255123,0.500426,0.908273,0.057921,0.636679


## Pass to GAMS and save to GDX file

In [6]:
# Define worspace and database
ws = gams.GamsWorkspace()
my_db = ws.add_database()

In [7]:
# Read in sets
g_tech = gmspy.list2set(my_db, tech, 'tech')
g_imps = gmspy.list2set(my_db, imps, 'imps', "All the environmental impacts")
g_cohort = gmspy.list2set(my_db, cohorts, 'cohort', "Year each car was built")

In [8]:
# Read in parameter
g_Y = gmspy.df2param(my_db, df=Y, domains=[g_tech], name='Y', comment="Total Final demand")

In [9]:
# Read in parameter
g_STOCK = gmspy.df2param(my_db, df=STOCK, domains=[g_tech, g_cohort], name='STOCK')

In [10]:
# Read in parameter
g_IMP  = gmspy.df2param(my_db, df=IMP, domains=[g_tech, g_imps, g_cohort], name='IMP')

In [11]:
# Save to gdx file
my_db.export('/tmp/demo.gdx')

## Inspect db or gdx files

In [12]:
# What is in this file?
gmspy.ls(gdx_filepath='/tmp/demo.gdx')

['tech', 'imps', 'cohort', 'Y', 'STOCK', 'IMP']

In [13]:
# What sets are in this my_db?
gmspy.ls(my_db, entity='Set')

['tech', 'imps', 'cohort']

## Extract from db or gdx files

In [14]:
# Extract a set from the database
extracted_set = gmspy.set2list('cohort', my_db)
extracted_set

['2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028']

In [15]:
# Extract a set from the gdx file
another_set = gmspy.set2list('tech', gdx_filepath='/tmp/demo.gdx')
another_set

['bev', 'icev']

In [16]:
# Extract parameter from GDX file
g_IMP_again = gmspy.param2series('IMP', gdx_filepath='/tmp/demo.gdx')
g_IMP_again

bev   gwp  2020    0.665257
           2021    0.996560
           2022    0.575023
           2023    0.224564
           2024    0.197891
           2025    0.298945
           2026    0.981139
           2027    0.400548
           2028    0.001846
icev  gwp  2020    0.414993
           2021    0.799778
           2022    0.410036
           2023    0.020946
           2024    0.255123
           2025    0.500426
           2026    0.908273
           2027    0.057921
           2028    0.636679
dtype: float64

In [17]:
# To get back to original shape of IMP, must be unstacked
g_IMP_again.unstack()

,,2020,2021,2022,2023,2024,2025,2026,2027,2028
bev,gwp,0.665257,0.996560,0.575023,0.224564,0.197891,0.298945,0.981139,0.400548,0.001846
icev,gwp,0.414993,0.799778,0.410036,0.020946,0.255123,0.500426,0.908273,0.057921,0.636679
